### Requirements

In [ ]:
!pip install -U python-dotenv matplotlib

### Import and load env

In [ ]:
from dotenv import load_dotenv
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import os
import sqlite3
import sys

load_dotenv()

VERBOSE = os.environ.get("VERBOSE")
MSG_DB_PATH = os.environ.get("MSG_DB_PATH")
FULL_DB_PATH = os.environ.get("FULL_DB_PATH")
START_TIMESTAMP = os.environ.get("START_TIMESTAMP")
END_TIMESTAMP = os.environ.get("END_TIMESTAMP")
TARGET_ID = os.environ.get("TARGET_ID")

if VERBOSE == "1":
    print("load_dotenv():")
    print(f"  {VERBOSE=}")
    print(f"  {MSG_DB_PATH=}")
    print(f"  {FULL_DB_PATH=}")
    print(f"  {START_TIMESTAMP=}")
    print(f"  {END_TIMESTAMP=}")
    print(f"  {TARGET_ID=}")

### Define helper methods

In [ ]:
def connect_db(path):
    try:
        conn = sqlite3.connect(path)
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting db: {e}")
        sys.exit(1)

def execute_query(conn, query, params=(), no_commit=False):
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        if not no_commit:
            conn.commit()
        if query.strip().lower().startswith("select"):
            return cursor.fetchall()
        else:
            return None
    except sqlite3.Error as e:
        print(f"Error executing SQL query: {e}")
        sys.exit(1)

### Create or load MSG db

In [ ]:
if not os.path.exists(MSG_DB_PATH):
    full_conn = connect_db(FULL_DB_PATH)

    query = f"""
    SELECT *
    FROM MSG
    WHERE Sequence >= {START_TIMESTAMP + '000'}
      AND Sequence <  {END_TIMESTAMP + '000'}
      AND StrTalker = '{TARGET_ID}';
    """
    msg = execute_query(full_conn, query)
    if not msg:
        print("error: no result from FULL_DB_PATH")
        sys.exit(1)
    print(f"Found {len(msg)} results.")
    full_conn.close()
    
    msg_conn = connect_db(MSG_DB_PATH)

    query = """
    CREATE TABLE MSG(
      localId INT,
      TalkerId INT,
      MsgSvrID INT,
      Type INT,
      SubType INT,
      IsSender INT,
      CreateTime INT,
      Sequence INT,
      StatusEx INT,
      FlagEx INT,
      Status INT,
      MsgServerSeq INT,
      MsgSequence INT,
      StrTalker TEXT,
      StrContent TEXT,
      DisplayContent TEXT,
      Reserved0 INT,
      Reserved1 INT,
      Reserved2 INT,
      Reserved3 INT,
      Reserved4 TEXT,
      Reserved5 TEXT,
      Reserved6 TEXT,
      CompressContent,
      BytesExtra,
      BytesTrans
    )
    """
    execute_query(msg_conn, query)
    query = "INSERT INTO MSG VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    for i, row in enumerate(msg):
        execute_query(msg_conn, query, row, no_commit=True)
        if i % 5000 == 0:
            msg_conn.commit()
    msg_conn.commit()
    print(f"{MSG_DB_PATH} is created.")

else:
    msg_conn = connect_db(MSG_DB_PATH)
    print(f"{MSG_DB_PATH} is found and loaded.")

---

### Report

今年我们一共发送了...

In [ ]:
res = execute_query(msg_conn, "SELECT COUNT(*) FROM MSG")
print(res[0][0])

...条消息。

In [ ]:
senders = ["Purple", "Blue"]
senders_cn = ["紫先生", "蓝小姐"]
colors = ["#b780ff", "#059af7"] # purple, blue
wedgeprops = dict(width=0.7, edgecolor='w', linewidth=3)

data = [
    execute_query(msg_conn, "SELECT COUNT(*) FROM MSG WHERE IsSender = 1")[0][0],
    execute_query(msg_conn, "SELECT COUNT(*) FROM MSG WHERE IsSender = 0")[0][0],
]
labels = [f"Purple: {data[0]}", f"Blue: {data[1]}"]

fig, ax = plt.subplots(figsize=(4, 4))
ax.pie(
    data,
    labels=labels,
    autopct="%.2f%%",
    colors=colors,
    wedgeprops=wedgeprops,
)

print(f"其中，{senders_cn[0]}发送了 {data[0]} 条，{senders_cn[1]}发送了 {data[1]} 条：")
plt.show()

在这么多条消息中里...

In [ ]:
types = {
    "Text":    (1, 0, "%"),             # 文本
    "Pic":     (3, 0, "%"),             # 图片
    "Video":   (43, 0, "%"),            # 视频
    "Voice":   (34, 0, "%"),            # 语音
    "Emoji_1": (47, 0, "%"),            # 商店表情
    "Emoji_2": (49, 8, "%"),            # 自定义表情
    "Call":    (50, 0, "%"),            # 通话
    "Trans_1": (49, 2000, "%"),         # 转账
    "Trans_2": (10000, 0, "发出红包%"), # 红包
    "Trans_3": (11000, 0, "%"),         # 红包
    "Tap":     (10000, 4, "%"),         # 拍一拍
}
data = {}
for t, p in types.items():
    data[t] = np.array([
        execute_query(msg_conn, "SELECT COUNT(*) FROM MSG WHERE IsSender = 1 AND Type = ? AND SubType = ? AND StrContent LIKE ?", p)[0][0],
        execute_query(msg_conn, "SELECT COUNT(*) FROM MSG WHERE IsSender = 0 AND Type = ? AND SubType = ? AND StrContent LIKE ?", p)[0][0],
    ])
data["Emoji"] = data["Emoji_1"] + data["Emoji_2"]
data["Trans"] = data["Trans_1"] + data["Trans_2"] + data["Trans_3"]
if VERBOSE == "1":
    pprint(data)

show = ["Text", "Pic", "Video", "Voice", "Emoji", "Call", "Tap", "Trans"]
data_show = np.array([data[i] for i in show])
if VERBOSE == "1":
    pprint(data_show)

x = np.arange(2)
width = 0.4
multiplier = 0

fig, ax = plt.subplots(figsize=(12, 4))
for i, d in enumerate(data_show):
    rects = ax.bar(x * width + i, d, width, color=colors)
    ax.bar_label(rects)
ax.set_xticks(np.arange(len(show)) + width/2, show)

def print_type_compare(key, desc):
    d = data[key]
    morer = 0 if d[0] > d[1] else 1
    lesser = 1 - morer
    pct = d[morer] * 100.0 / d[lesser] - 100
    print(f"{senders_cn[morer]}发出了更多的{desc}，达到了 {d[morer]} 条，比{senders_cn[lesser]}多出了 {pct:.2f}% ！")

print_type_compare("Text", "文字")
print_type_compare("Pic", "图片")
print_type_compare("Video", "视频")
print_type_compare("Voice", "语音")
print_type_compare("Emoji", "表情包")
print_type_compare("Call", "视频通话")
print_type_compare("Tap", "拍一拍")
print_type_compare("Trans", "红包和转账")
plt.show()